# Anna KaRNNa

In this notebook, we'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use. Here I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [2]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = sorted(set(text)) # Get character and seperate it out. Getting rid of duplicates
vocab_to_int = {c: i for i, c in enumerate(vocab)} # Maps the character to an integer
int_to_vocab = dict(enumerate(vocab)) # From Integers to characters
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32) # Integers of the text

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

And we can see the characters encoded as integers.

In [4]:
encoded[:100]

array([31, 64, 57, 72, 76, 61, 74,  1, 16,  0,  0,  0, 36, 57, 72, 72, 81,
        1, 62, 57, 69, 65, 68, 65, 61, 75,  1, 57, 74, 61,  1, 57, 68, 68,
        1, 57, 68, 65, 67, 61, 26,  1, 61, 78, 61, 74, 81,  1, 77, 70, 64,
       57, 72, 72, 81,  1, 62, 57, 69, 65, 68, 81,  1, 65, 75,  1, 77, 70,
       64, 57, 72, 72, 81,  1, 65, 70,  1, 65, 76, 75,  1, 71, 79, 70,  0,
       79, 57, 81, 13,  0,  0, 33, 78, 61, 74, 81, 76, 64, 65, 70],
      dtype=int32)

Since the network is working with individual characters, it's similar to a classification problem in which we are trying to predict the next character from the previous text.  Here's how many 'classes' our network has to pick from.

In [5]:
len(vocab)

#print(vocab) different kind of characters that are in the text

83

## Making training mini-batches

Here is where we'll make our mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

We start with our text encoded as integers in one long array in `encoded`. Let's create a function that will give us an iterator for our batches. I like using [generator functions](https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/) to do this. Then we can pass `encoded` into this function and get our batch generator.

The first thing we need to do is discard some of the text so we only have completely full batches. Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps. Then, to get the total number of batches, $K$, we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

After that, we need to split `arr` into $N$ sequences. You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences (`batch_size` below), let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

Now that we have this array, we can iterate through it to get our batches. The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `n_steps`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. 

The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `n_steps` wide.

> **Exercise:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [6]:
def get_batches(arr, batch_size, n_steps):
    '''Create a generator that returns batches of size
       batch_size x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    # Get the number of characters per batch and number of batches we can make
    characters_per_batch = batch_size * n_steps
    n_batches = int(len(arr) / characters_per_batch)
    chars_keep = int(characters_per_batch * n_batches)
    
    # Keep only enough characters to make full batches
    #
    arr = arr[:chars_keep]
    # print(arr.shape) 1985000
    
    # Reshape into batch_size rows
    size = (batch_size, -1)
    arr = arr.reshape(size)
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:,n:n+n_steps]
        # The targets, shifted by one
        y_temp = arr[:,n+1:n+n_steps+1]
        
        # The last batch will be one character short
        y = np.zeros(x.shape, dtype=x.dtype)
        y[:,:y_temp.shape[1]] = y_temp
        
        yield x, y

Now I'll make my data sets and we can check out what's going on here. Here I'm going to use a batch size of 10 and 50 sequence steps.

In [7]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

In [8]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[31 64 57 72 76 61 74  1 16  0]
 [ 1 57 69  1 70 71 76  1 63 71]
 [78 65 70 13  0  0  3 53 61 75]
 [70  1 60 77 74 65 70 63  1 64]
 [ 1 65 76  1 65 75 11  1 75 65]
 [ 1 37 76  1 79 57 75  0 71 70]
 [64 61 70  1 59 71 69 61  1 62]
 [26  1 58 77 76  1 70 71 79  1]
 [76  1 65 75 70  7 76 13  1 48]
 [ 1 75 57 65 60  1 76 71  1 64]]

y
 [[64 57 72 76 61 74  1 16  0  0]
 [57 69  1 70 71 76  1 63 71 65]
 [65 70 13  0  0  3 53 61 75 11]
 [ 1 60 77 74 65 70 63  1 64 65]
 [65 76  1 65 75 11  1 75 65 74]
 [37 76  1 79 57 75  0 71 70 68]
 [61 70  1 59 71 69 61  1 62 71]
 [ 1 58 77 76  1 70 71 79  1 75]
 [ 1 65 75 70  7 76 13  1 48 64]
 [75 57 65 60  1 76 71  1 64 61]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[55 63 69 22  6 76 45  5 16 35]
 [ 5 69  1  5 12 52  6  5 56 52]
 [48 29 12 61 35 35  8 64 76 78]
 [12  5 24 39 45 29 12 56  5 63]
 [ 5 29  6  5 29 78 28  5 78 29]
 [ 5 13  6  5 36 69 78 35 52 12]
 [63 76 12  5 18 52  1 76  5 58]
 [34  5 73 39  6  5 12 52 36  5]
 [ 6  5 29 78 12 79  6 61  5 59]
 [ 5 78 69 29 24  5  6 52  5 63]]

y
 [[63 69 22  6 76 45  5 16 35 35]
 [69  1  5 12 52  6  5 56 52 29]
 [29 12 61 35 35  8 64 76 78 28]
 [ 5 24 39 45 29 12 56  5 63 29]
 [29  6  5 29 78 28  5 78 29 45]
 [13  6  5 36 69 78 35 52 12 43]
 [76 12  5 18 52  1 76  5 58 52]
 [ 5 73 39  6  5 12 52 36  5 78]
 [ 5 29 78 12 79  6 61  5 59 63]
 [78 69 29 24  5  6 52  5 63 76]]
 ```
 although the exact numbers will be different. Check to make sure the data is shifted over one step for `y`.

## Building the model

Below is where you'll build the network. We'll break it up into parts so it's easier to reason about each bit. Then we can connect them up into the whole network.

<img src="assets/charRNN.png" width=500px>


### Inputs

First off we'll create our input placeholders. As usual we need placeholders for the training data and the targets. We'll also create a placeholder for dropout layers called `keep_prob`. This will be a scalar, that is a 0-D tensor. To make a scalar, you create a placeholder without giving it a size.

> **Exercise:** Create the input placeholders in the function below.

In [9]:
def build_inputs(batch_size, num_steps):
    ''' Define placeholders for inputs, targets, and dropout 
    
        Arguments
        ---------
        batch_size: Batch size, number of sequences per batch
        num_steps: Number of sequence steps in a batch
        
    '''
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32, shape=(batch_size, num_steps), name='inputs')
    targets = tf.placeholder(tf.int32, shape=(batch_size, num_steps), name='targets')
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

### LSTM Cell

Here we will create the LSTM cell we'll use in the hidden layer. We'll use this cell as a building block for the RNN. So we aren't actually defining the RNN here, just the type of cell we'll use in the hidden layer.

We first create a basic LSTM cell with

```python
lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
```

where `num_units` is the number of units in the hidden layers in the cell. Then we can add dropout by wrapping it with 

```python
tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
```
You pass in a cell and it will automatically add dropout to the inputs or outputs. Finally, we can stack up the LSTM cells into layers with [`tf.contrib.rnn.MultiRNNCell`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/rnn/MultiRNNCell). With this, you pass in a list of cells and it will send the output of one cell into the next cell. Previously with TensorFlow 1.0, you could do this

```python
tf.contrib.rnn.MultiRNNCell([cell]*num_layers)
```

This might look a little weird if you know Python well because this will create a list of the same `cell` object. However, TensorFlow 1.0 will create different weight matrices for all `cell` objects. But, starting with TensorFlow 1.1 you actually need to create new cell objects in the list. To get it to work in TensorFlow 1.1, it should look like

```python
def build_cell(num_units, keep_prob):
    lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    return drop
    
tf.contrib.rnn.MultiRNNCell([build_cell(num_units, keep_prob) for _ in range(num_layers)])
```

Even though this is actually multiple LSTM cells stacked on each other, you can treat the multiple layers as one cell.

We also need to create an initial cell state of all zeros. This can be done like so

```python
initial_state = cell.zero_state(batch_size, tf.float32)
```

Below, we implement the `build_lstm` function to create these LSTM cells and the initial state.

In [10]:
def build_cell(num_units, keep_prob):
    lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    return drop

def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    ### Build the LSTM Cell
    # Use a basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    # Add dropout to the cell outputs
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

### RNN Output

Here we'll create the output layer. We need to connect the output of the RNN cells to a full connected layer with a softmax output. The softmax output gives us a probability distribution we can use to predict the next character, so we want this layer to have size $C$, the number of classes/characters we have in our text.

If our input has batch size $N$, number of steps $M$, and the hidden layer has $L$ hidden units, then the output is a 3D tensor with size $N \times M \times L$. The output of each LSTM cell has size $L$, we have $M$ of them, one for each sequence step, and we have $N$ sequences. So the total size is $N \times M \times L$. 

We are using the same fully connected layer, the same weights, for each of the outputs. Then, to make things easier, we should reshape the outputs into a 2D tensor with shape $(M * N) \times L$. That is, one row for each sequence and step, where the values of each row are the output from the LSTM cells. We get the LSTM output as a list, `lstm_output`. First we need to concatenate this whole list into one array with [`tf.concat`](https://www.tensorflow.org/api_docs/python/tf/concat). Then, reshape it (with `tf.reshape`) to size $(M * N) \times L$.

Once we have the outputs reshaped, we can do the matrix multiplication with the weights. We need to wrap the weight and bias variables in a variable scope with `tf.variable_scope(scope_name)` because there are weights being created in the LSTM cells. TensorFlow will throw an error if the weights created here have the same names as the weights created in the LSTM cells, which they will be default. To avoid this, we wrap the variables in a variable scope so we can give them unique names.

> **Exercise:** Implement the output layer in the function below.

In [11]:
def build_output(lstm_output, in_size, out_size):
    ''' Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        lstm_output: List of output tensors from the LSTM layer
        in_size: Size of the input tensor, for example, size of the LSTM cells
        out_size: Size of this softmax layer. Number of chars.
    
    '''

    # Reshape output so it's a bunch of rows, one row for each step for each sequence.
    # Concatenate lstm_output over axis 1 (the columns)
    seq_output = tf.concat(lstm_output, axis=1)
    # Reshape seq_output to a 2D tensor with lstm_size columns
    x = tf.reshape(seq_output, [-1, in_size])
    
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        # Create the weight and bias variables here
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    # Use softmax to get the probabilities for predicted characters
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits

### Training loss

Next up is the training loss. We get the logits and targets and calculate the softmax cross-entropy loss. First we need to one-hot encode the targets, we're getting them as encoded characters. Then, reshape the one-hot targets so it's a 2D tensor with size $(M*N) \times C$ where $C$ is the number of classes/characters we have. Remember that we reshaped the LSTM outputs and ran them through a fully connected layer with $C$ units. So our logits will also have size $(M*N) \times C$.

Then we run the logits and targets through `tf.nn.softmax_cross_entropy_with_logits` and find the mean to get the loss.

>**Exercise:** Implement the loss calculation in the function below.

In [12]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    
    # One-hot encode targets and reshape to match logits, one row per sequence per step
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped =  tf.reshape(y_one_hot, logits.get_shape())
    
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_reshaped, logits=logits)
    loss = tf.reduce_mean(loss)
    
    return loss

### Optimizer

Here we build the optimizer. Normal RNNs have have issues gradients exploding and disappearing. LSTMs fix the disappearance problem, but the gradients can still grow without bound. To fix this, we can clip the gradients above some threshold. That is, if a gradient is larger than that threshold, we set it to the threshold. This will ensure the gradients never grow overly large. Then we use an AdamOptimizer for the learning step.

In [13]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

### Build the network

Now we can put all the pieces together and build a class for the network. To actually run data through the LSTM cells, we will use [`tf.nn.dynamic_rnn`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/nn/dynamic_rnn). This function will pass the hidden and cell states across LSTM cells appropriately for us. It returns the outputs for each LSTM cell at each step for each sequence in the mini-batch. It also gives us the final LSTM state. We want to save this state as `final_state` so we can pass it to the first LSTM cell in the the next mini-batch run. For `tf.nn.dynamic_rnn`, we pass in the cell and initial state we get from `build_lstm`, as well as our input sequences. Also, we need to one-hot encode the inputs before going into the RNN. 

> **Exercise:** Use the functions you've implemented previously and `tf.nn.dynamic_rnn` to build the network.

In [14]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Run each sequence step through the RNN with tf.nn.dynamic_rnn 
        outputs, state = tf.nn.dynamic_rnn(cell, inputs=x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss = build_loss(logits=self.logits, targets=self.targets, lstm_size=lstm_size, num_classes=num_classes)
        self.optimizer = build_optimizer(loss=self.loss, learning_rate=learning_rate, grad_clip=grad_clip)

## Hyperparameters

Here are the hyperparameters for the network.

* `batch_size` - Number of sequences running through the network in one pass.
* `num_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lstm_size` - The number of units in the hidden layers.
* `num_layers` - Number of hidden LSTM layers to use
* `learning_rate` - Learning rate for training
* `keep_prob` - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `lstm_size` and `num_layers`. I would advise that you always use `num_layers` of either 2/3. The `lstm_size` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `lstm_size` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

In [15]:
batch_size = 10         # Sequences per batch
num_steps = 50          # Number of sequence steps per batch
lstm_size = 128         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.01    # Learning rate
keep_prob = 0.5         # Dropout keep probability

## Time for training

This is typical training code, passing inputs and targets into the network, then running the optimizer. Here we also get back the final LSTM state for the mini-batch. Then, we pass that state back into the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I save a checkpoint.

Here I'm saving checkpoints with the format

`i{iteration number}_l{# hidden layer units}.ckpt`

> **Exercise:** Set the hyperparameters above to train the network. Watch the training loss, it should be consistently dropping. Also, I highly advise running this on a GPU.

In [16]:
epochs = 20
# Print losses every N interations
print_every_n = 50

# Save every N iterations
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            if (counter % print_every_n == 0):
                end = time.time()
                print('Epoch: {}/{}... '.format(e+1, epochs),
                      'Training Step: {}... '.format(counter),
                      'Training loss: {:.4f}... '.format(batch_loss),
                      '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Epoch: 1/20...  Training Step: 50...  Training loss: 3.1196...  0.0682 sec/batch
Epoch: 1/20...  Training Step: 100...  Training loss: 2.7180...  0.0688 sec/batch
Epoch: 1/20...  Training Step: 150...  Training loss: 2.4783...  0.0684 sec/batch
Epoch: 1/20...  Training Step: 200...  Training loss: 2.4889...  0.0680 sec/batch
Epoch: 1/20...  Training Step: 250...  Training loss: 2.3661...  0.0686 sec/batch
Epoch: 1/20...  Training Step: 300...  Training loss: 2.2450...  0.0684 sec/batch
Epoch: 1/20...  Training Step: 350...  Training loss: 2.1439...  0.0681 sec/batch
Epoch: 1/20...  Training Step: 400...  Training loss: 2.2203...  0.0677 sec/batch
Epoch: 1/20...  Training Step: 450...  Training loss: 2.2106...  0.0674 sec/batch
Epoch: 1/20...  Training Step: 500...  Training loss: 2.2845...  0.0680

Epoch: 2/20...  Training Step: 4800...  Training loss: 1.6490...  0.0674 sec/batch
Epoch: 2/20...  Training Step: 4850...  Training loss: 1.6688...  0.0665 sec/batch
Epoch: 2/20...  Training Step: 4900...  Training loss: 1.7419...  0.0679 sec/batch
Epoch: 2/20...  Training Step: 4950...  Training loss: 1.5906...  0.0669 sec/batch
Epoch: 2/20...  Training Step: 5000...  Training loss: 1.7027...  0.0661 sec/batch
Epoch: 2/20...  Training Step: 5050...  Training loss: 1.5235...  0.0659 sec/batch
Epoch: 2/20...  Training Step: 5100...  Training loss: 1.5498...  0.0660 sec/batch
Epoch: 2/20...  Training Step: 5150...  Training loss: 1.7176...  0.0672 sec/batch
Epoch: 2/20...  Training Step: 5200...  Training loss: 1.8072...  0.0665 sec/batch
Epoch: 2/20...  Training Step: 5250...  Training loss: 1.6188...  0.0658 sec/batch
Epoch: 2/20...  Training Step: 5300...  Training loss: 1.8117...  0.0667 sec/batch
Epoch: 2/20...  Training Step: 5350...  Training loss: 1.6456...  0.0664 sec/batch
Epoc

Epoch: 3/20...  Training Step: 9750...  Training loss: 1.4652...  0.0674 sec/batch
Epoch: 3/20...  Training Step: 9800...  Training loss: 1.6685...  0.0677 sec/batch
Epoch: 3/20...  Training Step: 9850...  Training loss: 1.5269...  0.0660 sec/batch
Epoch: 3/20...  Training Step: 9900...  Training loss: 1.6845...  0.0676 sec/batch
Epoch: 3/20...  Training Step: 9950...  Training loss: 1.6709...  0.0667 sec/batch
Epoch: 3/20...  Training Step: 10000...  Training loss: 1.5343...  0.0669 sec/batch
Epoch: 3/20...  Training Step: 10050...  Training loss: 1.5814...  0.0664 sec/batch
Epoch: 3/20...  Training Step: 10100...  Training loss: 1.6787...  0.0678 sec/batch
Epoch: 3/20...  Training Step: 10150...  Training loss: 1.7409...  0.0672 sec/batch
Epoch: 3/20...  Training Step: 10200...  Training loss: 1.8290...  0.0666 sec/batch
Epoch: 3/20...  Training Step: 10250...  Training loss: 1.6208...  0.0663 sec/batch
Epoch: 3/20...  Training Step: 10300...  Training loss: 1.8030...  0.0674 sec/bat

Epoch: 4/20...  Training Step: 14650...  Training loss: 1.5645...  0.0670 sec/batch
Epoch: 4/20...  Training Step: 14700...  Training loss: 1.4791...  0.0661 sec/batch
Epoch: 4/20...  Training Step: 14750...  Training loss: 1.5916...  0.0666 sec/batch
Epoch: 4/20...  Training Step: 14800...  Training loss: 1.5733...  0.0679 sec/batch
Epoch: 4/20...  Training Step: 14850...  Training loss: 1.6418...  0.0667 sec/batch
Epoch: 4/20...  Training Step: 14900...  Training loss: 1.6042...  0.0655 sec/batch
Epoch: 4/20...  Training Step: 14950...  Training loss: 1.7620...  0.0697 sec/batch
Epoch: 4/20...  Training Step: 15000...  Training loss: 1.5748...  0.0694 sec/batch
Epoch: 4/20...  Training Step: 15050...  Training loss: 1.7353...  0.0679 sec/batch
Epoch: 4/20...  Training Step: 15100...  Training loss: 1.5231...  0.0670 sec/batch
Epoch: 4/20...  Training Step: 15150...  Training loss: 1.5022...  0.0670 sec/batch
Epoch: 4/20...  Training Step: 15200...  Training loss: 1.7386...  0.0675 se

Epoch: 5/20...  Training Step: 19550...  Training loss: 1.5699...  0.0660 sec/batch
Epoch: 5/20...  Training Step: 19600...  Training loss: 1.5875...  0.0662 sec/batch
Epoch: 5/20...  Training Step: 19650...  Training loss: 1.5049...  0.0675 sec/batch
Epoch: 5/20...  Training Step: 19700...  Training loss: 1.7771...  0.0667 sec/batch
Epoch: 5/20...  Training Step: 19750...  Training loss: 1.6884...  0.0665 sec/batch
Epoch: 5/20...  Training Step: 19800...  Training loss: 1.8527...  0.0665 sec/batch
Epoch: 5/20...  Training Step: 19850...  Training loss: 1.8892...  0.0671 sec/batch
Epoch: 6/20...  Training Step: 19900...  Training loss: 1.6861...  0.0680 sec/batch
Epoch: 6/20...  Training Step: 19950...  Training loss: 1.5350...  0.0670 sec/batch
Epoch: 6/20...  Training Step: 20000...  Training loss: 1.3546...  0.0666 sec/batch
Epoch: 6/20...  Training Step: 20050...  Training loss: 1.5607...  0.0678 sec/batch
Epoch: 6/20...  Training Step: 20100...  Training loss: 1.5398...  0.0675 se

...

Epoch: 15/20...  Training Step: 58500...  Training loss: 1.6769...  0.0671 sec/batch
Epoch: 15/20...  Training Step: 58550...  Training loss: 1.4437...  0.0671 sec/batch
Epoch: 15/20...  Training Step: 58600...  Training loss: 1.5686...  0.0685 sec/batch
Epoch: 15/20...  Training Step: 58650...  Training loss: 1.4980...  0.0663 sec/batch
Epoch: 15/20...  Training Step: 58700...  Training loss: 1.3819...  0.0667 sec/batch
Epoch: 15/20...  Training Step: 58750...  Training loss: 1.5109...  0.0665 sec/batch
Epoch: 15/20...  Training Step: 58800...  Training loss: 1.5366...  0.0693 sec/batch
Epoch: 15/20...  Training Step: 58850...  Training loss: 1.5405...  0.0670 sec/batch
Epoch: 15/20...  Training Step: 58900...  Training loss: 1.4370...  0.0668 sec/batch
Epoch: 15/20...  Training Step: 58950...  Training loss: 1.3862...  0.0666 sec/batch
Epoch: 15/20...  Training Step: 59000...  Training loss: 1.7169...  0.0681 sec/batch
Epoch: 15/20...  Training Step: 59050...  Training loss: 1.5169..

Epoch: 16/20...  Training Step: 63350...  Training loss: 1.4405...  0.0668 sec/batch
Epoch: 16/20...  Training Step: 63400...  Training loss: 1.5005...  0.0667 sec/batch
Epoch: 16/20...  Training Step: 63450...  Training loss: 1.6744...  0.0671 sec/batch
Epoch: 16/20...  Training Step: 63500...  Training loss: 1.4657...  0.0672 sec/batch
Epoch: 17/20...  Training Step: 63550...  Training loss: 1.4264...  0.0677 sec/batch
Epoch: 17/20...  Training Step: 63600...  Training loss: 1.4634...  0.0670 sec/batch
Epoch: 17/20...  Training Step: 63650...  Training loss: 1.4746...  0.0659 sec/batch
Epoch: 17/20...  Training Step: 63700...  Training loss: 1.3664...  0.0673 sec/batch
Epoch: 17/20...  Training Step: 63750...  Training loss: 1.4390...  0.0666 sec/batch
Epoch: 17/20...  Training Step: 63800...  Training loss: 1.5149...  0.0663 sec/batch
Epoch: 17/20...  Training Step: 63850...  Training loss: 1.5441...  0.0662 sec/batch
Epoch: 17/20...  Training Step: 63900...  Training loss: 1.5388..

Epoch: 18/20...  Training Step: 68200...  Training loss: 1.5626...  0.0675 sec/batch
Epoch: 18/20...  Training Step: 68250...  Training loss: 1.4094...  0.0667 sec/batch
Epoch: 18/20...  Training Step: 68300...  Training loss: 1.3770...  0.0677 sec/batch
Epoch: 18/20...  Training Step: 68350...  Training loss: 1.6039...  0.0667 sec/batch
Epoch: 18/20...  Training Step: 68400...  Training loss: 1.5658...  0.0663 sec/batch
Epoch: 18/20...  Training Step: 68450...  Training loss: 1.5117...  0.0665 sec/batch
Epoch: 18/20...  Training Step: 68500...  Training loss: 1.5644...  0.0669 sec/batch
Epoch: 18/20...  Training Step: 68550...  Training loss: 1.4423...  0.0677 sec/batch
Epoch: 18/20...  Training Step: 68600...  Training loss: 1.4217...  0.0665 sec/batch
Epoch: 18/20...  Training Step: 68650...  Training loss: 1.3492...  0.0671 sec/batch
Epoch: 18/20...  Training Step: 68700...  Training loss: 1.4614...  0.0668 sec/batch
Epoch: 18/20...  Training Step: 68750...  Training loss: 1.3981..

Epoch: 19/20...  Training Step: 73050...  Training loss: 1.5333...  0.0657 sec/batch
Epoch: 19/20...  Training Step: 73100...  Training loss: 1.5432...  0.0672 sec/batch
Epoch: 19/20...  Training Step: 73150...  Training loss: 1.4500...  0.0666 sec/batch
Epoch: 19/20...  Training Step: 73200...  Training loss: 1.4727...  0.0659 sec/batch
Epoch: 19/20...  Training Step: 73250...  Training loss: 1.4450...  0.0670 sec/batch
Epoch: 19/20...  Training Step: 73300...  Training loss: 1.4761...  0.0672 sec/batch
Epoch: 19/20...  Training Step: 73350...  Training loss: 1.4979...  0.0677 sec/batch
Epoch: 19/20...  Training Step: 73400...  Training loss: 1.7377...  0.0689 sec/batch
Epoch: 19/20...  Training Step: 73450...  Training loss: 1.5317...  0.0671 sec/batch
Epoch: 19/20...  Training Step: 73500...  Training loss: 1.4831...  0.0677 sec/batch
Epoch: 19/20...  Training Step: 73550...  Training loss: 1.4317...  0.0676 sec/batch
Epoch: 19/20...  Training Step: 73600...  Training loss: 1.4722..

Epoch: 20/20...  Training Step: 77900...  Training loss: 1.4885...  0.0678 sec/batch
Epoch: 20/20...  Training Step: 77950...  Training loss: 1.5779...  0.0672 sec/batch
Epoch: 20/20...  Training Step: 78000...  Training loss: 1.3830...  0.0679 sec/batch
Epoch: 20/20...  Training Step: 78050...  Training loss: 1.3544...  0.0681 sec/batch
Epoch: 20/20...  Training Step: 78100...  Training loss: 1.4890...  0.0665 sec/batch
Epoch: 20/20...  Training Step: 78150...  Training loss: 1.3768...  0.0677 sec/batch
Epoch: 20/20...  Training Step: 78200...  Training loss: 1.4992...  0.0678 sec/batch
Epoch: 20/20...  Training Step: 78250...  Training loss: 1.6002...  0.0672 sec/batch
Epoch: 20/20...  Training Step: 78300...  Training loss: 1.3987...  0.0674 sec/batch
Epoch: 20/20...  Training Step: 78350...  Training loss: 1.6233...  0.0671 sec/batch
Epoch: 20/20...  Training Step: 78400...  Training loss: 1.3809...  0.0688 sec/batch
Epoch: 20/20...  Training Step: 78450...  Training loss: 1.5890..

#### Saved checkpoints

Read up on saving and loading checkpoints here: https://www.tensorflow.org/programmers_guide/variables

In [17]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i79400_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i59600_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i59800_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i60000_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i60200_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i60400_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i60600_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i60800_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i61000_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i61200_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i61400_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i61600_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i61800_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i62000_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i62200_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i62400_l128.ckpt"
all_model_checkpoint_paths: "checkpoints/i62600_l128.ckpt"
al

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [18]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [19]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

Here, pass in the path to a checkpoint and sample from the network.

In [20]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints/i79400_l128.ckpt'

In [21]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i79400_l128.ckpt
Fary arm they were not some and and thought and all, where, if she said in a bist, and something of him in him of the staid worned.

The ways that he had not comant on tight on
her that at the
plit so at a bong. The chaid at her, was no saw and talked of so on a state, and he seemed his women his windres of harment
and to to say. "Hive where thought's been only of what she's a
look of when I should no stop, I'm not be sorry and then it are now out in
tastings to be begin about the taken with the pairing in it to himself.

It has not so of the preperous of it,"
he said, strunged his wathar of
harrest. "I do not tend a sinter and heard there is that sat the position. The offering."

She could be between him. But her from the
took worse and handed his sound and at him.

"I censed this went with the cried in waiting and his wife.
It's no to the strength at the constant. What was this shool tin thrast to
the
whole senter

In [22]:
checkpoint = 'checkpoints/i200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i200_l512.ckpt


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for checkpoints/i200_l512.ckpt
	 [[Node: save/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2/tensor_names, save/RestoreV2/shape_and_slices)]]
	 [[Node: save/RestoreV2/_3 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_8_save/RestoreV2", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'save/RestoreV2', defined at:
  File "/home/ubuntu/anaconda3/envs/intro-to-rnns/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/envs/intro-to-rnns/lib/python3.6/runpy.py", line

In [ ]:
checkpoint = 'checkpoints/i600_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

In [ ]:
checkpoint = 'checkpoints/i1200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)